In [1]:
import pypsa 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from custom_constraints import set_operational_limits

In [2]:
model_file = pd.ExcelFile("../data/model_file.xlsx")
model_setup = (
    pd.read_excel(
        model_file, 
        sheet_name="model_setup",
        index_col=[0])
        .loc["grid-expansion"]
)


In [3]:
#n0 = pypsa.Network('../networks/elec_grid-2040_30-supply_redz.nc')
n0 = pypsa.Network('../networks/pre_grid-2030-2040_11-supply_redz_lcopt_LC-300SEG.nc')

INFO:pypsa.io:Imported network pre_grid-2030-2040_11-supply_redz_lcopt_LC-300SEG.nc has buses, carriers, generators, global_constraints, lines, loads, storage_units


In [4]:
n0.optimize.create_model(multi_investment_periods=True)
set_operational_limits(n0, model_file, model_setup)
# m.parameters = m.parameters.drop("snapshots")
# m.to_netcdf("test_network.nc")

In [6]:
n0.optimize.solve_model(solver_name='xpress', solver_options = {"lpflags":4, "crossover":0})

INFO:linopy.model: Solve problem using Xpress solver
INFO:linopy.model:Solver options:
 - lpflags: 4
 - crossover: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 9/9 [00:00<00:00, 122.78it/s]
INFO:linopy.io: Writing time: 0.67s


Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/home/pklein/anaconda3/envs/pypsa-earth-updated/lib/python3.10/site-packages/xpress/license/community-xpauth.xpr')
 
Reading Problem linopy-problem-9kwrholo
Problem Statistics
      234647 (      0 spare) rows
       88570 (      0 spare) structural columns
      438890 (      0 spare) non-zero elements
MIP Entity Statistics
           0 entities        0 sets        0 set members
FICO Xpress v9.1.2, Hyper, solve started 8:18:12, Nov 13, 2023
Heap usage: 66MB (peak 104MB, 2567KB system)
Minimizing LP linopy-problem-9kwrholo using up to 20 threads and up to 125GB memory, with these control settings:
OUTPUTLOG = 1
CROSSOVER = 0
LPFLAGS = 4
Original problem has:
    234647 rows        88570 cols     

Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: lp_infeas



('warning', 'infeasible')

In [7]:
n0.model.constraints["max-ocgt_diesel-week-2040"]

Constraint `max-ocgt_diesel-week-2040` (week: 48):
--------------------------------------------------
[1]: +11 Generator-p[(2040, Timestamp('2040-01-02 06:00:00')), Eastern Cape-ocgt_diesel_IPP] + 13 Generator-p[(2040, Timestamp('2040-01-02 17:00:00')), Eastern Cape-ocgt_diesel_IPP] + 12 Generator-p[(2040, Timestamp('2040-01-03 06:00:00')), Eastern Cape-ocgt_diesel_IPP] ... -5.357 Generator-p_nom[Free State-ocgt_diesel-2040] - 3.826 Generator-p_nom[Free State-ocgt_diesel-2040] - 5.357 Generator-p_nom[Free State-ocgt_diesel-2040]   ≤ 67938.0
[2]: +12 Generator-p[(2040, Timestamp('2040-01-09 07:00:00')), Eastern Cape-ocgt_diesel_IPP] + 14 Generator-p[(2040, Timestamp('2040-01-09 19:00:00')), Eastern Cape-ocgt_diesel_IPP] + 8 Generator-p[(2040, Timestamp('2040-01-10 09:00:00')), Eastern Cape-ocgt_diesel_IPP] ... -5.74 Generator-p_nom[Free State-ocgt_diesel-2040] - 3.444 Generator-p_nom[Free State-ocgt_diesel-2040] - 14.54 Generator-p_nom[Free State-ocgt_diesel-2040]     ≤ 67536.0
[3]: +11

In [ ]:
n0.model.variables.get_label_position(107)

In [ ]:
n0.generators.p_nom_opt.groupby(n0.generators.carrier).sum()

In [ ]:
n0.storage_units.query("carrier == 'phs'").p_nom_opt

In [ ]:
n0.generators_t.p[n0.generators[n0.generators.carrier=="wind"].index].sum()/1e6

In [ ]:
n0.generators_t.p[n0.generators[n0.generators.carrier=="coal"].index].sum().sum()/1e6

In [ ]:
n0.generators_t.p[n0.generators[n0.generators.carrier=="solar_pv"].index].sum().sum()/1e6

In [ ]:
n0.loads_t.p_set.sum().sum()/1e6

In [ ]:
n0.storage_units.query("carrier=='phs'")

In [ ]:
n0.statistics()

In [ ]:
n0.loads_t.p_set.sum(axis=1).plot()

In [ ]:
n0.generators_t.p_max_pu.mean()

In [ ]:
n0.generators.query("p_nom_extendable == False")

In [ ]:
n0.generators_t.p.mean()

In [ ]:
#n0.buses.loc["Northern Cape", "nom_max_wind"] = 2000
n0.buses.loc["Hydra Cluster", "nom_max_solar_pv"] = 1000
n0.buses.loc["Gauteng", "nom_min_nuclear"] = 2200

In [ ]:
#n0.export_to_netcdf('../networks/solved_elec_grid-expansion_11-supply_redz.nc')

In [ ]:
model_setup = "base"
model_file = pd.ExcelFile('../data/model_file.xlsx')

ext_max_build = (
    pd.read_excel(
        model_file, 
        sheet_name='extendable_max_build',
        index_col=[0,1,2,3])
).loc[model_setup]
ext_max_build.replace("unc", np.inf, inplace=True)

# global max build


# bus-carrier max_build








ext_min_build = (
    pd.read_excel(
        model_file, 
        sheet_name='extendable_min_build',
        index_col=[0,1,2])
).loc[model_setup]
ext_min_build.replace(0, np.nan, inplace=True)

# build_constraints = (pd.read_excel('../data/model_file.xlsx', 
#                             sheet_name='new_build_limits',
#                             index_col=[0,1,2])).loc[model_setup['new_build_limits']]

# max_build = build_constraints.loc['max_installed_limit'].fillna(100000)
# min_build = build_constraints.loc['min_installed_limit']
         


In [ ]:
ext_max_build.loc[("11-supply","Western Cape")]

In [ ]:
n0.generators.p_nom_opt.groupby(n0.generators.carrier).sum()

In [ ]:
n0.storage_units.p_nom_opt.groupby(n0.storage_units.carrier).sum()

In [ ]:
n0.statistics()